In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Paths
data_dir = "C:/Users/ASUS/Documents/KOLNEY/MACHINE LEARNING/Final Project/dataset"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "validation")  

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Class info
class_names = train_dataset.classes
num_classes = len(class_names)

print("Classes:", class_names)
print("Total classes:", num_classes)


Classes: ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'not ingredient', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']
Total classes: 37


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)

# Replace final layer
model.fc = nn.Linear(model.fc.in_features, num_classes)  

model = model.to(device)


c:\Users\ASUS\Documents\KOLNEY\MACHINE LEARNING\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\Documents\KOLNEY\MACHINE LEARNING\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [5]:
epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss/len(val_loader):.4f} | "
          f"Val Acc: {100 * correct/total:.2f}%")


c:\Users\ASUS\Documents\KOLNEY\MACHINE LEARNING\.venv\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10 | Train Loss: 1.6222 | Val Loss: 0.3798 | Val Acc: 88.09%
Epoch 2/10 | Train Loss: 0.5667 | Val Loss: 0.3003 | Val Acc: 89.20%
Epoch 3/10 | Train Loss: 0.4099 | Val Loss: 0.2722 | Val Acc: 90.30%
Epoch 4/10 | Train Loss: 0.3078 | Val Loss: 0.2083 | Val Acc: 94.18%
Epoch 5/10 | Train Loss: 0.2281 | Val Loss: 0.1828 | Val Acc: 94.46%
Epoch 6/10 | Train Loss: 0.1916 | Val Loss: 0.1990 | Val Acc: 92.80%
Epoch 7/10 | Train Loss: 0.1662 | Val Loss: 0.2184 | Val Acc: 94.18%
Epoch 8/10 | Train Loss: 0.1441 | Val Loss: 0.2105 | Val Acc: 93.35%
Epoch 9/10 | Train Loss: 0.1284 | Val Loss: 0.1990 | Val Acc: 93.91%
Epoch 10/10 | Train Loss: 0.1172 | Val Loss: 0.1812 | Val Acc: 95.57%


In [6]:
torch.save(model, "fruit_veg_classifier_2.pth")

In [7]:
from PIL import Image


In [8]:
def predict_image(image_path):
    model.eval()

    img = Image.open(image_path)
    img_tensor = val_transforms(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        _, pred = torch.max(output, 1)

    return class_names[pred.item()]
